In [17]:
# Load libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, r2_score
from scipy.stats import linregress

In [19]:
# Read Dataset (Any classification dataset of your choice from kaggle, should be tubular)
import pandas as pd

df = pd.read_csv("Raisin_Dataset.csv")
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [21]:
df.shape

(900, 8)

In [22]:
df.isnull().sum()

,0
Area,0
MajorAxisLength,0
MinorAxisLength,0
Eccentricity,0
ConvexArea,0
Extent,0
Perimeter,0
Class,0


In [23]:
df.duplicated().sum()

0

In [25]:
y=df.Class.values
x=df.drop(["Class"],axis=1)
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=labelencoder.fit_transform(y)

In [26]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
x=mms.fit_transform(x)

In [29]:
from sklearn.model_selection import train_test_split
x_train_class, x_test_class, y_train_class, y_test_class = train_test_split(x, y, test_size=0.2, random_state = 42)

In [31]:
# Define a simple classification neural network model
model_cls = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_class.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
# Compile the model
model_cls.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_cls.fit(x_train_class, y_train_class, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6965 - loss: 0.6478
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8154 - loss: 0.5907 
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8308 - loss: 0.5223 
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8461 - loss: 0.4741 
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8506 - loss: 0.4178 
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8538 - loss: 0.3928 
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8631 - loss: 0.3849 
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8388 - loss: 0.3993 
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8506 - loss: 0.3846
Epoch 10/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8716 - loss: 0.3498 


In [35]:
# Evaluate the model
accuracy = model_cls.evaluate(x_test_class, y_test_class, verbose=0)[1]
print("Accuracy:", accuracy)

Accuracy: 0.855555534362793


In [36]:
# Evaluate the model
y_pred_prob_class = model_cls.predict(x_test_class)
y_pred_class = np.round(y_pred_prob_class).flatten()

print("Classification Report:")
print(classification_report(y_test_class, y_pred_class))
print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.83      0.85        86
           1       0.85      0.88      0.86        94

    accuracy                           0.86       180
   macro avg       0.86      0.85      0.85       180
weighted avg       0.86      0.86      0.86       180

Confusion Matrix:
[[71 15]
 [11 83]]


In [39]:
## Deploy the above network with dropout layer

# Define a classification neural network model with Dropout layers
model_cls = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_class.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model, This is binary_crossentropy loss function
model_cls.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_cls.fit(x_train_class, y_train_class, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
accuracy = model_cls.evaluate(x_test_class, y_test_class, verbose=0)[1]
print("Accuracy:", accuracy)

# Predict and evaluate
y_pred_prob_class = model_cls.predict(x_test_class)
y_pred_class = np.round(y_pred_prob_class).flatten()

print("Classification Report:")
print(classification_report(y_test_class, y_pred_class))
print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.5815 - loss: 0.6671
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6626 - loss: 0.6488 
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6836 - loss: 0.6178
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7400 - loss: 0.5853 
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7972 - loss: 0.5275 
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7970 - loss: 0.4956 
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8009 - loss: 0.4723 
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8169 - loss: 0.4427 
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8211 - loss: 0.4296 
Epoch 10/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7894 - loss: 0.4656 


Accuracy: 0.8611111044883728
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.85      0.85        86
           1       0.86      0.87      0.87        94

    accuracy                           0.86       180
   macro avg       0.86      0.86      0.86       180
weighted avg       0.86      0.86      0.86       180

Confusion Matrix:
[[73 13]
 [12 82]]


In [43]:
## Deploy the above network with different loss fucntion and different optimizer¶

# Define a classification neural network model with Dropout layers
model_cls = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_class.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Compile the model with a different loss function and optimizer
model_cls.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model_cls.fit(x_train_class, y_train_class, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
accuracy = model_cls.evaluate(x_test_class, y_test_class, verbose=0)[1]
print("Accuracy:", accuracy)

# Predict and evaluate
y_pred_prob_class = model_cls.predict(x_test_class)
y_pred_class = np.round(y_pred_prob_class).flatten()

print("Classification Report:")
print(classification_report(y_test_class, y_pred_class))
print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.4953 - loss: 0.2503
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5698 - loss: 0.2280  
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6754 - loss: 0.2095 
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7098 - loss: 0.2078 
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7368 - loss: 0.1900
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7592 - loss: 0.1876
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7804 - loss: 0.1749 
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7621 - loss: 0.1689 
Epoch 9/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7906 - loss: 0.1657 
Epoch 10/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7906 - loss: 0.1551
Accuracy: 0.8500000238418579
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Classification Report:
              precision    recall  f1-score   support

           0     

In [44]:
# Deploy the above network with different epochs size and batch size

# Define a classification neural network model with Dropout layers
model_cls = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(x_train_class.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with a different loss function and optimizer
model_cls.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model_cls.fit(x_train_class, y_train_class, epochs=20, batch_size=64, verbose=1)

# Evaluate the model
accuracy = model_cls.evaluate(x_test_class, y_test_class, verbose=0)[1]
print("Accuracy:", accuracy)

# Predict and evaluate
y_pred_prob_class = model_cls.predict(x_test_class)
y_pred_class = np.round(y_pred_prob_class).flatten()

print("Classification Report:")
print(classification_report(y_test_class, y_pred_class))
print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.5506 - loss: 0.2438
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6105 - loss: 0.2305 
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6763 - loss: 0.2164 
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6901 - loss: 0.2155 
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7474 - loss: 0.1954 
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7384 - loss: 0.1895 
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7538 - loss: 0.1805  
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7758 - loss: 0.1695 
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8147 - loss: 0.1570 
Epoch 10/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7981 - loss: 0.1634 
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7994 - loss: 0.1531 
Epoch 12/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7910 - l

In [74]:
!pip install tensorflow


In [75]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential


In [76]:
# Read Dataset (Any regression dataset of your choice from kaggle, should be tubular)
import pandas as pd

df1 = pd.read_csv("insurance.csv")
df1.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [77]:
df1.shape

(1338, 7)

In [78]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [79]:
df1.isnull().sum()

,0
age,0
sex,0
bmi,0
children,0
smoker,0
region,0
charges,0


In [80]:
df1.duplicated().sum()


1

In [81]:
df1 = df1.drop_duplicates()


In [82]:
df1.shape

(1337, 7)

In [83]:
##Converting objects labels into categorical
df1[['sex', 'smoker', 'region']] = df1[['sex', 'smoker', 'region']].astype('category')
df1.dtypes

,0
age,int64
sex,category
bmi,float64
children,int64
smoker,category
region,category
charges,float64


In [84]:
##Converting category labels into numerical using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
label.fit(df1.sex.drop_duplicates())
df1.sex = label.transform(df1.sex)
label.fit(df1.smoker.drop_duplicates())
df1.smoker = label.transform(df1.smoker)
label.fit(df1.region.drop_duplicates())
df1.region = label.transform(df1.region)
df1.dtypes

,0
age,int64
sex,int64
bmi,float64
children,int64
smoker,int64
region,int64
charges,float64


In [85]:
from sklearn.model_selection import train_test_split as holdout
from sklearn import metrics
x = df1.drop(['charges'], axis = 1)
y = df1['charges']
x_train_reg, x_test_reg, y_train_reg, y_test_reg = holdout(x, y, test_size=0.2, random_state=0)

In [89]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Define a simple regression neural network model
model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_reg.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(x_train_reg, y_train_reg, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(x_test_reg, y_test_reg)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 292616288.0000 - mae: 12658.3779 - val_loss: 364315840.0000 - val_mae: 14153.8857
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 291157888.0000 - mae: 12772.7051 - val_loss: 358560576.0000 - val_mae: 13958.6904
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 274661312.0000 - mae: 12103.1309 - val_loss: 342795168.0000 - val_mae: 13410.5186
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 286696544.0000 - mae: 11953.7979 - val_loss: 312730304.0000 - val_mae: 12305.6689
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 265568096.0000 - mae: 11158.6641 - val_loss: 269778400.0000 - val_mae: 10691.0156
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 193378976.0000 - mae: 8739.4492 - val_loss: 223014976.0000 - val_mae: 9081.1992
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 152971552.0000 - mae: 7307.9048 - val_loss: 185632480.0000 - val_mae: 8226.2305
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━

In [90]:
## Dropout layers

import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Define a simple regression neural network model with Dropout
model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_reg.shape[1],)),
    layers.Dropout(0.3),  # Dropout with 30% rate
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% rate
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(x_train_reg, y_train_reg, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(x_test_reg, y_test_reg)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 294099520.0000 - mae: 12761.0449 - val_loss: 363326496.0000 - val_mae: 14121.3672
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 283069280.0000 - mae: 12334.6699 - val_loss: 353809920.0000 - val_mae: 13798.2695
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 266391664.0000 - mae: 11624.0586 - val_loss: 328666496.0000 - val_mae: 12904.8379
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 262674464.0000 - mae: 11382.8838 - val_loss: 282992736.0000 - val_mae: 11167.9248
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 192923648.0000 - mae: 8805.2021 - val_loss: 226835760.0000 - val_mae: 9132.9170
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 165758800.0000 - mae: 7802.8579 - val_loss: 181129120.0000 - val_mae: 8118.0562
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 153293392.0000 - mae: 8049.5020 - val_loss: 161997216.0000 - val_mae: 8617.1641
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━

In [92]:
## use other optmizer and loss function
import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Define a simple regression neural network model with Dropout
model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_reg.shape[1],)),
    layers.Dropout(0.3),  # Dropout with 30% rate
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% rate
    layers.Dense(1)  # Output layer for regression
])

# Compile the model with RMSprop optimizer and Huber loss
model.compile(optimizer='RMSprop', loss=tf.keras.losses.Huber(), metrics=['mae'])

# Train the model
history = model.fit(x_train_reg, y_train_reg, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(x_test_reg, y_test_reg)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 12982.9561 - mae: 12983.4561 - val_loss: 14085.6885 - val_mae: 14086.1885
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 12120.5762 - mae: 12121.0762 - val_loss: 13856.7217 - val_mae: 13857.2217
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 11988.7744 - mae: 11989.2744 - val_loss: 13443.7568 - val_mae: 13444.2568
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12455.5381 - mae: 12456.0381 - val_loss: 12807.7373 - val_mae: 12808.2373
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11703.0283 - mae: 11703.5283 - val_loss: 11944.4209 - val_mae: 11944.9209
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10508.1689 - mae: 10508.6689 - val_loss: 10904.2500 - val_mae: 10904.7500
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9562.4297 - mae: 9562.9297 - val_loss: 9859.0176 - val_mae: 9859.5176
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8504.2090 - mae: 8504.7090 - val

In [93]:
## Use other epoch and batch size

import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Define a simple regression neural network model with Dropout
model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(x_train_reg.shape[1],)),
    layers.Dropout(0.3),  # Dropout with 30% rate
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),  # Dropout with 30% rate
    layers.Dense(1)  # Output layer for regression
])

# Compile the model with RMSprop optimizer and Huber loss
model.compile(optimizer='RMSprop', loss=tf.keras.losses.Huber(), metrics=['mae'])

# Train the model with fewer epochs and a smaller batch size
history = model.fit(x_train_reg, y_train_reg, epochs=30, batch_size=8, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(x_test_reg, y_test_reg)
print(f'Test Loss: {loss:.4f}, Test MAE: {mae:.4f}')


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 13095.6846 - mae: 13096.1846 - val_loss: 13805.7910 - val_mae: 13806.2910
Epoch 2/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 11840.7227 - mae: 11841.2227 - val_loss: 12430.1660 - val_mae: 12430.6660
Epoch 3/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 10517.2031 - mae: 10517.7031 - val_loss: 10099.9932 - val_mae: 10100.4932
Epoch 4/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 8105.8091 - mae: 8106.3091 - val_loss: 8555.2129 - val_mae: 8555.7129
Epoch 5/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7836.7612 - mae: 7837.2612 - val_loss: 8240.4287 - val_mae: 8240.9287
Epoch 6/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7539.9409 - mae: 7540.4409 - val_loss: 8135.9214 - val_mae: 8136.4214
Epoch 7/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7870.6641 - mae: 7871.1641 - val_loss: 8143.2715 - val_mae: 8143.7715
Epoch 8/30
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7269.1050 - mae: 7269.6050 -